In [ ]:
# Follows the DBNL Quickstart at https://docs.dbnl.com/get-started/quickstart
!pip install --upgrade dbnl

In [ ]:
import dbnl
import pandas as pd
import pyarrow.parquet as pq
from datetime import UTC, datetime, timedelta

# Make sure your version matches the docs at https://docs.dbnl.com/
print("dbnl version:", dbnl.__version__)

In [ ]:
# Login to DBNL (using default Sandbox url)
dbnl.login(
    api_url="http://localhost:8080/api",
    api_token="", # found at http://localhost:8080/tokens
)

In [ ]:
# Create a new project
project = dbnl.get_or_create_project(
    name="Quickstart Demo",
    schedule="daily",  # How often DBNL analyzes new data
    default_llm_model_name="quickstart_model" # From step (2) in quickstart
)

In [ ]:
# Load 14 days of real LLM conversation logs from public S3 bucket
base_path = "s3://dbnl-demo-public/outing_agent_log_data"
s3_files = [f"{base_path}/day_{i:02d}.parquet" for i in range(1, 15)]
day_dfs = [pq.read_table(f).to_pandas(types_mapper=pd.ArrowDtype, ignore_metadata=True) for f in s3_files]

# Adjust timestamps to current time so data appears recent
delta = datetime.now(tz=UTC) - day_dfs[-1]["timestamp"].max()
delta = timedelta(days=round(delta / timedelta(days=1)))
for df in day_dfs:
    df["timestamp"] = df["timestamp"] + delta

In [ ]:
# Upload the data, DBNL needs at least 7 days to establish behavioral baselines
print("Uploading data...")
print(f"See status at: http://localhost:8080/ns/{project.namespace_id}/projects/{project.id}/status")
for idx, day_df in enumerate(day_dfs):
    print(f"{idx + 1} / {len(day_dfs)} publishing log data")
    data_start_t = min(day_df['timestamp']).replace(hour=0, minute=0, second=0, microsecond=0)
    data_end_t = data_start_t + timedelta(days=1)
    try:
        dbnl.log(
            project_id=project.id,
            data_start_time=data_start_t,
            data_end_time=data_end_t,
            data=day_df,
        )
    except Exception as e:
        if "Data already exists" in str(e):
            continue
        raise
    
print("You can now explore your data in DBNL!")
print(f"http://localhost:8080/ns/{project.namespace_id}/projects/{project.id}")